# Andmete puhastamine

Siin töötleme varasemas etapis kogutud toorandmeid (`toorandmed_aasta.csv`).
Faili leiad projekti ÜHISEST repositooriumist. 

**Sisend:** `../andmed/toorandmed_aasta.csv`

**Väljund:** `../andmed/andmed_aasta.csv`

### Samm 0: Teekide laadimine ja failiteede seadistamine 
Laeme vajalikud Pythoni teegid ja fikseerime sisend- ja väljundfailide asukohad. Loeme sisse puhastamata andmed.

In [19]:
import pandas as pd
import numpy as np
import os

# Konfigureerime Pandase sätteid loetavuse huvides
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Sisend- ja väljundfailid
INPUT_FILE = '../andmed/toorandmed_aasta.csv'
OUTPUT_FILE = '../andmed/andmed_aasta.csv'


if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE, low_memory=False)
    print(f"Toorandmed loetud. Andmestiku suurus: {df.shape[0]} rida, {df.shape[1]} veergu.")
else:
    # Kui faili pole, loome testimiseks tühja DataFrame'i või viskame vea
    raise FileNotFoundError(f"Viga: Sisendfaili '{INPUT_FILE}' ei leitud! Kontrolli failiteed.")

# Kiire pilk andmestruktuurile


print("Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.")

df.head(2)

Toorandmed loetud. Andmestiku suurus: 3156 rida, 229 veergu.
Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.


,course_uuid,code,title__en,version__title__en,credits,overview__description__en,version__overview__description__en,overview__learning_outcomes_text_en,uuid,state__code,state__et,state__en,last_update,title__et,general__input_languages,general__structural_unit_shares,general__type__code,general__type__et,general__type__en,general__structural_units,additional_info__duration_in_semesters,additional_info__assessment_scale__code,additional_info__assessment_scale__et,additional_info__assessment_scale__en,additional_info__is_vota_course,additional_info__is_continuous_learning_course,overview__description__et,overview__objectives,overview__learning_outcomes,_actions__is_editable,_actions__is_version_creatable,_actions__is_deletable,overview__objectives_text_en,version__uuid,version__code,version__state__code,version__state__et,version__state__en,version__last_update,version__title__et,version__credits,version__credit_type,version__target__year__code,version__target__year__et,version__target__year__en,version__target__semester__code,version__target__semester__et,version__target__semester__en,version__target__study_type__code,version__target__study_type__et,version__target__study_type__en,version__target__language__code,version__target__language__et,version__target__language__en,version__target__part__code,version__target__part__et,version__target__part__en,version__target__faculty__code,version__target__faculty__name__en,version__target__faculty__name__et,version__target__faculty__address,version__target__faculty__city,version__target__faculty__level,version__target__faculty__academic,version__target__faculty__supports_continuous_learning,version__target__course_main_structural_unit__code,version__target__course_main_structural_unit__name__en,version__target__course_main_structural_unit__name__et,version__target__course_main_structural_unit__webpage_url,version__target__course_main_structural_unit__address,version__target__course_main_structural_unit__city,version__target__course_main_structural_unit__street,version__target__course_main_structural_unit__zip,version__target__course_main_structural_unit__level,version__target__course_main_structural_unit__academic,version__target__course_main_structural_unit__supports_continuous_learning,version__target__course_main_structural_unit__parent_code,version__target__label__en,version__target__label__et,version__general__year__code,version__general__year__et,version__general__year__en,version__general__input_languages,version__general__structural_unit_shares,version__general__type__code,version__general__type__et,version__general__type__en,version__additional_info__is_vota_course,version__additional_info__is_continuous_learning_course,version__additional_info__study_levels,version__additional_info__hours__lecture,version__additional_info__hours__practice,version__additional_info__hours__seminar,version__additional_info__hours__colloquium,version__additional_info__hours__web_learning,version__additional_info__hours__independent_work,version__additional_info__hours__individual_work,version__additional_info__hours__internship,version__additional_info__is_enlight_course,version__overview__study_languages,version__overview__description__et,version__overview__objectives,version__overview__learning_outcomes,version__grading__independent_work_assessments,version__grading__grade_preconditions__et,version__grading__grade_evaluation__et,version__grading__debt_elimination__et,version__grading__assessment_scale__code,version__grading__assessment_scale__et,version__grading__assessment_scale__en,version__resources__web_based__code,version__resources__web_based__et,version__resources__web_based__en,version__resources__website_url,version__resources__is_moodle_connected,version__resources__learning_environments,version__resources__materials,version__participants__lecturers,version__schedule__weeks__et,version__schedule__entries,version__parent_uuid,version__parent_code,version__parent_credits,version___action

### Samm 1: eemaldame andmed, mida me ei taha

Rakendame minimaalsed eelfiltrid, mis olid seminaris kokku lepitud:
- eemaldame ained, mille kestus on üle 1 semestri;
- eemaldame kaitsmisega seotud ained (kui pealkiri viitab kaitsmisele);
- proovime jätta ainult päevaõppe, kui vastav tunnus on olemas.

### Samm 2: Puhastatud veergude lisamine

Valime välja huvipakkuvad veerud ning puhastame need. Antud sammu juures seisneb puhastamine õige veeru valimises - juhul kui huvipakkuv sisu võib olla kahes erinevas veerus, eelistame aine konkreetse versiooni infot.


Täpsem kirjeldus aine üldine info vs versiooni info probleemist:
* **Probleem:** Meil on topelt veerge eeldatavasti sama infoga (üldine aine info vs aine konkreetse versiooni info). Nt eestikeelne pealkiri võiks peituda nii `title__et` kui ka `version__title__et` veerus.
* **Lahendus:** Eelistame alati `version__` prefiksiga veerge, sest need kajastavad tegelikku aineprogrammi. Kui see puudub, võtame üldise info.

In [20]:
# --- Samm 1: Eelfiltrid ---
df_work = df.copy()
rows_before = len(df_work)

# 1) eemaldame > 1 semestri ained (kui veerg olemas)
if 'additional_info__duration_in_semesters' in df_work.columns:
    duration = pd.to_numeric(df_work['additional_info__duration_in_semesters'], errors='coerce')
    df_work = df_work[(duration.isna()) | (duration <= 1)]

# 2) eemaldame kaitsmisega seotud ained (heuristika ET+EN pealkirjade põhjal)
title_cols = [c for c in ['version__title__et', 'title__et', 'version__title__en', 'title__en'] if c in df_work.columns]
if title_cols:
    title_text = df_work[title_cols].fillna('').astype(str).agg(' '.join, axis=1)
    df_work = df_work[~title_text.str.contains(r'kaitsm|defen[cs]e', case=False, regex=True)]

# 2b) lisaks eemaldame kaitsmise viited hindamise kirjeldusest (kui olemas)
if 'version__grading__grade_evaluation__et' in df_work.columns:
    eval_text = df_work['version__grading__grade_evaluation__et'].fillna('').astype(str)
    df_work = df_work[~eval_text.str.contains(r'kaitsm', case=False, regex=True)]

# 3) ainult päevaõpe (kui tunnus olemas)
if 'version__target__study_type__et' in df_work.columns:
    study_type = df_work['version__target__study_type__et'].fillna('').astype(str)
    day_mask = study_type.str.contains('päeva', case=False, regex=False)
    if day_mask.any():
        df_work = df_work[day_mask]

print(f"Samm 1: Eelfiltrid rakendatud. Ridasi enne: {rows_before}, pärast: {len(df_work)}")

# --- Samm 2: Võtame ALLES ainult kokkulepitud tunnused ---
selected_raw_cols = [
    'code',
    'credits',
    'additional_info__prerequisites',
    'overview__description__en',
    'overview__description__et',
    'title__en',
    'title__et',
    'version__target__language__et',
    'version__target__semester__et',
    'additional_info__assessment_scale__et',
    'version__overview__description__en',
    'version__overview__description__et',
    'version__overview__study_languages',
    'version__target__course_main_structural_unit__city',
    'version__additional_info__study_levels',
    'version__grading__grade_evaluation__et',
]

# Lisame puuduvad veerud NaN-na, et väljundi skeem oleks stabiilne
for col in selected_raw_cols:
    if col not in df_work.columns:
        df_work[col] = np.nan

df_resolved = df_work[selected_raw_cols].copy()

print('Samm 2: Valitud tunnused lukustatud (ainult kokkulepitud 16 veergu).')
print('Valitud veerud:', len(df_resolved.columns))
display(pd.Series(df_resolved.columns, name='selected_features'))

Samm 1: Eelfiltrid rakendatud. Ridasi enne: 3156, pärast: 3058
Samm 2: Valitud tunnused lukustatud (ainult kokkulepitud 16 veergu).
Valitud veerud: 16


0                                                   code
1                                                credits
2                         additional_info__prerequisites
3                              overview__description__en
4                              overview__description__et
5                                              title__en
6                                              title__et
7                          version__target__language__et
8                          version__target__semester__et
9                  additional_info__assessment_scale__et
10                    version__overview__description__en
11                    version__overview__description__et
12                    version__overview__study_languages
13    version__target__course_main_structural_unit__city
14                version__additional_info__study_levels
15                version__grading__grade_evaluation__et
Name: selected_features, dtype: str


### Samm 3: JSON väljade parsimine

See koodiplokk tegeleb tehniliste JSON-formaadis veergude teisendamisega inimloetavaks tekstiks. Eesmärk on eraldada keerulisest struktuurist vaid oluline sisu (nt nimed või nädalapäevad) ja vormistada see komadega eraldatud loeteluks.

Näide teisendamisest (õppejõud):
* Enne (Toores JSON): ``'[{"person_name": "Mari Mets", "id": 101}, {"person_name": "Jaan Kask", "id": 102}]'``
* Pärast (Puhastatud): ``"Jaan Kask, Mari Mets"``

In [21]:
# Samm 3: Selles versioonis me lisatöötlust ei tee, sest nõue on
# hoida alles ainult kokkulepitud 16 toorveergu.
print('Samm 3: JSON eraldi töötlus vahele jäetud (nõue: only selected features).')
print(f'Andmestik jätkub kujul: {df_resolved.shape[0]} rida, {df_resolved.shape[1]} veergu.')

Samm 3: JSON eraldi töötlus vahele jäetud (nõue: only selected features).
Andmestik jätkub kujul: 3058 rida, 16 veergu.


### Samm 4 (Valikuline): Info koondamine mitmest allikast

Kui vajalik info on mitmes veerus, saame need ühendada. Hetkel teeme seda näiteks hindamisinfoga.

In [22]:
# Samm 4: Selles versioonis ei koonda lisainfot, sest väljundis tohib olla
# ainult kokkulepitud tunnuste nimekiri.
print('Samm 4: Lisakoondamist ei tehta (nõue: only selected features).')
print(f'Veergude arv on jätkuvalt: {df_resolved.shape[1]}')

Samm 4: Lisakoondamist ei tehta (nõue: only selected features).
Veergude arv on jätkuvalt: 16


### Samm 5: Lõplik viimistlus

Valime lõplikud veerud, lisame hard-filtri tunnused ja valmistame RAG jaoks ühendatud tekstivälja.

In [23]:
# Samm 5: Lõplik viimistlus
df_final = df_resolved[selected_raw_cols].copy()

print(f"Lõplik ridade arv: {len(df_final)}, lõplik veergude arv: {len(df_final.columns)}")
print('Samm 5: Lõplik andmestik koostatud.')
print('Kontroll: kas väljundis on ainult kokkulepitud tunnused?')
print(list(df_final.columns) == selected_raw_cols)

Lõplik ridade arv: 3058, lõplik veergude arv: 16
Samm 5: Lõplik andmestik koostatud.
Kontroll: kas väljundis on ainult kokkulepitud tunnused?
True


### Samm 6: Salvestamine ja kodutöö väljundid

Selles sammus näitame kodutöö esitamiseks vajalikud väljundid:
- puuduvate väärtuste hulk igas veerus;
- kategooriliste tunnuste enim levinud väärtused;
- "Sissejuhatus andmeteadusesse" aine puhastatud kujul loetavalt;
- tunnus `kirjeldus`, mis ühendab ühe aine koguinfo tekstiks;
- tunnuse `kirjeldus` tähemärkide arvu statistika.

In [24]:


# 1) Puuduvate väärtuste hulk igas veerus
print('\nPuuduvate väärtuste arv veergude kaupa:')
missing = df_final.isna().sum().sort_values(ascending=False)
display(missing)

# 2) Kategooriliste tunnuste enim levinud väärtused
print('\nKategooriliste tunnuste top-väärtused:')
categorical_cols = [
    'version__target__language__et',
    'version__target__semester__et',
    'version__additional_info__study_levels',
    'version__overview__study_languages',
    'version__target__course_main_structural_unit__city',
    'additional_info__assessment_scale__et',
]

for col in categorical_cols:
    if col in df_final.columns:
        print(f"\nVeerg: {col}")
        display(df_final[col].value_counts(dropna=False).head(5))

# 3) Näita AINULT ainet koodiga LTAT.02.002
target_code = 'LTAT.02.002'
sample_df = df_final[df_final['code'].astype(str).str.strip().eq(target_code)].copy()

if not sample_df.empty:
    sample_pretty = sample_df.head(1).T
    sample_pretty.columns = ['väärtus']
    print(f'\nNäidis: aine koodiga {target_code} (puhastatud kuju)')
    display(sample_pretty)
else:
    print(f'\nAinet koodiga {target_code} ei leitud andmestikust.')

# 4) Lisame tunnuse 'kirjeldus' (kogu aine info ühes tekstis)
report_cols = list(df_final.columns)
df_report = df_final.copy()

def build_course_description(row):
    parts = []
    for col in report_cols:
        value = row.get(col)
        if pd.notna(value) and str(value).strip() != '':
            parts.append(f"{col}: {value}")
    return "\n".join(parts)

df_report['kirjeldus'] = df_report.apply(build_course_description, axis=1)
df_report['kirjeldus_pikkus'] = df_report['kirjeldus'].str.len()

print('\nTunnuse kirjeldus pikkuse statistika:')
display(df_report['kirjeldus_pikkus'].describe())

print(f"\nKokku tähemärke: {df_report['kirjeldus_pikkus'].sum()}")

# 5) Kiire eelvaade
print('\nLõppandmestiku eelvaade:')
display(df_final.head(3))

# Salvesta puhastatud andmestik (ainult kokkulepitud 16 veergu)
df_final.to_csv(OUTPUT_FILE, index=False)
print(f"Samm 6: Andmestik salvestatud faili '{OUTPUT_FILE}'.")


Puuduvate väärtuste arv veergude kaupa:


additional_info__prerequisites                        2506
version__target__course_main_structural_unit__city    1128
version__grading__grade_evaluation__et                 369
version__additional_info__study_levels                  38
version__overview__description__et                       1
version__overview__description__en                       1
version__overview__study_languages                       1
code                                                     0
version__target__language__et                            0
title__et                                                0
title__en                                                0
overview__description__et                                0
overview__description__en                                0
credits                                                  0
version__target__semester__et                            0
additional_info__assessment_scale__et                    0
dtype: int64


Kategooriliste tunnuste top-väärtused:

Veerg: version__target__language__et


version__target__language__et
eesti keel      2252
inglise keel     806
Name: count, dtype: int64


Veerg: version__target__semester__et


...


Tunnuse kirjeldus pikkuse statistika:


count     3058.000000
mean      2765.159908
std       1339.559387
min        615.000000
25%       1823.500000
50%       2451.000000
75%       3353.000000
max      11073.000000
Name: kirjeldus_pikkus, dtype: float64


Kokku tähemärke: 8455859

Lõppandmestiku eelvaade:


,code,credits,additional_info__prerequisites,overview__description__en,overview__description__et,title__en,title__et,version__target__language__et,version__target__semester__et,additional_info__assessment_scale__et,version__overview__description__en,version__overview__description__et,version__overview__study_languages,version__target__course_main_structural_unit__city,version__additional_info__study_levels,version__grading__grade_evaluation__et
0,OIEO.06.046,6.00,NaN,"The course covers the main topics of private international law (PIL), its terminology and develo...","Kursuse raames käsitletakse rahvusvahelise eraõiguse põhiteemasid, terminoloogiat ja kujunemist,...",Private International Law,Rahvusvaheline eraõigus,eesti keel,kevad,"Eristav (A, B, C, D, E, F, mi)","The course covers the main topics of private international law (PIL), its terminology and develo...","Kursuse raames käsitletakse rahvusvahelise eraõiguse põhiteemasid, terminoloogiat ja kujunemist,...","[{""code"": ""et"", ""en"": ""Estonian"", ""et"": ""eesti keel""}]",Tartu linn,"[{""code"": ""master"", ""en"": ""master's studies"", ""et"": ""magistriõpe""}]",100% kirjalik eksam.
1,KKSB.05.092,3.00,NaN,Subject Practicing in the work environment the students expand their knowledge in the field of p...,Aines Praktiseerimine töökeskkonnas laiendab üliõpilane oma silmaringi füsioteraapia valdkonnas ...,Practice in the Work Environment,Praktiseerimine töökeskkonnas,eesti keel,kevad,"Eristamata (arv, m.arv, mi)",Subject Practicing in the work environment the students expand their knowledge in the field of p...,Aines Praktiseerimine töökeskkonnas laiendab üliõpilane oma silmaringi füsioteraapia valdkonnas ...,"[{""code"": ""et"", ""en"": ""Estonian"", ""et"": ""eesti keel""}]",Tartu linn,"[{""code"": ""bachelor"", ""en"": ""bachelor's studies"", ""et"": ""bakalaureuseõpe""}]",NaN
2,ARKI.02.030,2.25,"[{""code"": ""AR00.00.016"", ""credits"": 17, ""last_update"": ""2024-02-05T11:04:36"", ""required"": true, ...","Diagnostic and treatment principles, methods and tactics of surgical diseases in children. The c...","Lastekirurgiliste haiguste diagnoosimise ja ravi printsiibid, metoodika ja taktika. Antud aine m...",Paediatric Surgery,Lastekirurgia,inglise keel,sügis,"Eristamata (arv, m.arv, mi)","Diagnostic and treatment principles, methods and tactics of surgical diseases in children. The c...","Lastekirurgiliste haiguste diagnoosimise ja ravi printsiibid, metoodika ja taktika. Antud aine m...","[{""code"": ""en"", ""en"": ""English"", ""et"": ""inglise keel""}]",Tartu linn,"[{""code"": ""bachelor_master"", ""en"": ""integrated bachelor's and master's studies"", ""et"": ""integree...","A part of the complec exam ""Surgery"""


Samm 6: Andmestik salvestatud faili '../andmed/andmed_aasta.csv'.
